<a href="https://colab.research.google.com/github/misioskyyy/codingGigants/blob/main/tagi_gier_wspoldzielone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1.Wczytaj zbiór danych
2 Zlicz ile rekordów znajduje się w zbiorze danych. Ile różnych gier znajduje
się w zbiorze danych i ile razy każda z tych gier się pojawia
3. Zlicz ile jest unikalnych tagów oraz ile tagów jest przypisanych tylko do
jednej gry, a ile do kilku
4. Wyświetl 20 najczęściej występujących tagów wraz z liczbą wystąpień

In [33]:
import json
import pandas as pd

df = pd.read_json('/content/tagi_gier_wspoldzielone.json')

with open('/content/tagi_gier_wspoldzielone.json') as f:
  data = json.load(f)

df.head()


,label,tags
0,minecraft,"[rankingi, sandbox, pistolet, event, smok, dia..."
1,fortnite,"[tryb solo, snajper, online, autobus bojowy, m..."
2,fortnite,"[sezon, kopanie, misje, drużyna, fortyfikacje,..."
3,roblox,"[czat, karabin, tryb kreatywny, klocki, oddzia..."
4,lol,"[przetrwanie, snajper, zombie, esport, skiny, ..."


In [34]:
count = {}

for game in data:
  label = game["label"]
  if label not in count:
    count[label] = 0
  count[label] += 1

print(len(data))
print(len(count))

for label in count:
  print(label, count[label])


#tag_count = {}
#for game in data:
#  label = game["label"]
#  for tag in game["tags"]:
#    if tag not in tag_count:
#      tag_count[tag] +=1



300
5
minecraft 60
fortnite 60
roblox 60
lol 60
cs 60


In [36]:
import json, random, math
def load_data(path):
  with open(path, encoding="utf-8") as f:
    return json.load(f)

def train_test_split(data, test_ratio = 0.20):
  random.shuffle(data)
  cut = int(len(data) * (1-test_ratio))

  print(f'Ilość próbek zbioru treningowego: {len(data[:cut])}')
  print(f"Ilość próbek zbioru testowego: {len(data[cut])}")
  return data[:cut], data[cut:]

def build_vocabulary(train):
  vocab = set()
  for rec in train:
    vocab.update(rec["tags"])
  print(vocab)
  return vocab

def train_navie_bayes(train, vocab, alpha = 0.1):
  class_count = {}
  word_counts = {}
  total_words = {}

  for rec in train:
    c = rec['label']
    class_count[c] = class_count.get(c,0)+1
    word_counts.setdefault(c, {})
    total_words.setdefault(c,0)

    for tag in rec["tags"]:
      word_counts[c][tag] = word_counts[c].get(tag, 0)+1
      total_words[c] += 1

    model = {
        "class_counts": class_count,
        "word_counts": word_counts,
        "vocab": vocab,
        "alpha": alpha,
        "total_docs": len(train)
    }
    print(model)
    return model


  def log_prob(model, rec, class_name):
    logp = math.log(model["class_counts"][class_name]/model["total_docs"])
    v = len(model['vocab'])
    a = model['alpha']

    for tag in rec['tags']:
      wc = model['word_counts'][class_name].get(tag,0)
      logp += math.log((wc+a)/ (model["total_words"][class_name]+a*v))

    return logp

  def predict(model, rec):
    besr_class, best_log = None, -1e99
    for c in model["class_counts"]:
      lp = log_prob(model, rec, c)
      if lp > best_log:
        best_class, best_log = c, lp
      return best_class


In [37]:
def main():
  path = "tagi_gier_wspoldzielone.json"
  data = load_data(path)
  train, test = train_test_split(data, test_ratio = 0.2)
  vocab = build_vocabulary(data)
  model = train_navie_bayes(train,vocab, alpha = 1.0)

main()

Ilość próbek zbioru treningowego: 240
Ilość próbek zbioru testowego: 2
{'kopanie', 'hardcore', 'rankingi', 'wyzwania', 'esport', 'pistolet', 'crafting', 'budowanie', 'kolekcje', 'burza', 'strzał w głowę', 'biomy', 'lama', 'przetrwanie', 'baron', 'czat', 'obby', 'nexus', 'sezon', 'studio', 'v-bucksy', 'roleplay', 'oddziały', 'lua', 'emotki', 'loot', 'bomba', 'event', 'tryb', 'arena', 'parkour', 'redstone', 'survival', 'skiny', 'klocki', 'zombie', 'rankedy', 'nether', 'strzelanie', 'autobus bojowy', 'dust2', 'multiplayer', 'robuxy', 'awp', 'prywatny serwer', 'questy', 'creepery', 'tryb kreatywny', 'sandbox', 'karabin', 'snajper', 'waluta', 'runy', 'wydarzenia', 'fortyfikacje', 'drużyna', 'tryb solo', 'mapa', 'smok', 'flash', 'misje', 'diamenty', 'symulator', 'tickrate', 'duety', 'online', 'bohaterowie', 'aktualizacja', 'sklep', 'slurp', 'brokhaven', 'dżungla', 'ulti', 'battle'}
{'class_counts': {'roblox': 1}, 'word_counts': {'roblox': {'lua': 1, 'parkour': 1, 'survival': 1, 'studio': 1, 